In [43]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import random
from datasets import load_dataset
device = "cuda" # the device to load the model onto

In [36]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen2-7B-Instruct",  # path2Qwen2
    torch_dtype="auto",
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("Qwen2-7B-Instruct")


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.32s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
def chat(system, prompt, model=model, tokenizer=tokenizer):

    messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt}
        ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=768
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [46]:
import json
datasets='GAOKAO_math'
content = load_dataset('json', data_files=f'Eval/dataset/{datasets}.jsonl',split='train')
content

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 10
})

In [47]:
idx = random.randint(0, len(content))
instruction, question, answer = content[idx].values()
prompt = '{question}'.format(question=question)
response = chat(instruction, prompt)
print("The question is:", prompt)
print("The prediction is: ", response)
print("The answer is: ", answer)

The question is: 问题是:已知向量 ā = (0, 1), b̄ = (2, x)，若 b̄ ⊥ (b̄ - 4ā)，则 x = (  ), 选项是:A: -2, B: -1, C: 1, D: 2, 您的答案是:
The prediction is:  要解决这个问题，我们首先需要理解题目中的条件 "b̄ ⊥ (b̄ - 4ā)"，这表示向量 b̄ 与向量 (b̄ - 4ā) 是垂直的。两个向量垂直意味着它们的点积为0。

给定向量 ā = (0, 1) 和 b̄ = (2, x)，我们可以计算出 (b̄ - 4ā) = (2 - 4*0, x - 4*1) = (2, x - 4)。

两个向量的点积公式为：\[ \vec{a} \cdot \vec{b} = a_xb_x + a_yb_y \]

所以，对于向量 b̄ 和 (b̄ - 4ā)，我们有：

\[ b̄ \cdot (b̄ - 4ā) = 2*2 + x*(x-4) = 0 \]

\[ 4 + x^2 - 4x = 0 \]

整理得：

\[ x^2 - 4x + 4 = 0 \]

这是一个完全平方公式，可以写为：

\[ (x - 2)^2 = 0 \]

因此，解得：

\[ x = 2 \]

所以，正确答案是 D: 2。
The answer is:  D
